In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ratings = pd.read_csv('./drive/My Drive/ratings.csv')
movies = pd.read_csv('./drive/My Drive/movies.csv')

pd.set_option('display.max_columns', 6)
pd.set_option('display.width', 300)

movie_ratings = pd.merge(ratings, movies, on='movieId')
print(movie_ratings)

In [ ]:
title_user = movie_ratings.pivot_table('rating', index='userId', columns='title')
title_user.fillna(0, inplace=True)
print(title_user)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 유저와 유저 간의 유사도
user_based_collab = cosine_similarity(title_user, title_user)
print(user_based_collab)

In [ ]:
user_based_collab = pd.DataFrame(user_based_collab, index=title_user.index, columns=title_user.index)
print(user_based_collab)

In [ ]:
# 1번째 유저와 유사한 10명의 유저를 뽑는다
print(user_based_collab[1].sort_values(ascending=False)[:10])

In [ ]:
# 가장 유사한 유저를 뽑아서 해당 유저가 본 영화 추천
user = user_based_collab[1].sort_values(ascending=False)[:10].index[1]
result = title_user.query(f"userId == {user}").sort_values(ascending=False, by=user, axis=1)
print(result)

In [ ]:
# 1번 유저와 유사한 유저들 9명을 뽑아서, 그 유저들이 어떤 영화에 대해서 부여한 평점에
# 유사도만큼의 가중치를 부여해서 이걸 토대로 1번 유저가 부여할 평점을 계산/예측
# 가중치 -> 유저 9명 유사도의 합 중에서 해당 유저가 차지하는 유사도
user_index_list = user_based_collab[1].sort_values(ascending=False)[:10].index.tolist()
user_weight_list = user_based_collab[1].sort_values(ascending=False)[:10].tolist()
print(user_index_list)
print(user_weight_list)
print(sum(user_weight_list)-1)

In [ ]:
# 유사한 유저들조차도 평점을 부여하지 않았다면
# 1번 유저가 부여할 평점을 예측할 수 없다...
movie_title = 'Dark Knight, The (2008)'
weighted_sum = []
for i in range(1, 10):
    weighted_sum.append(title_user[movie_title][user_index_list[i]] * user_weight_list[i]/(sum(user_weight_list)-1))
print(sum(weighted_sum))

# 아이템 기반

In [ ]:
user_title = movie_ratings.pivot_table('rating', index='title', columns='userId')
user_title.fillna(0, inplace=True)
print(user_title)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 영화와 영화 간의 유사도
item_based_collab = cosine_similarity(user_title, user_title)
print(item_based_collab)

In [ ]:
item_based_collab = pd.DataFrame(item_based_collab, index=user_title.index, columns=user_title.index)
print(item_based_collab)

In [ ]:
print(item_based_collab['Avatar (2009)'].sort_values(ascending=False)[:10])

# 잠재 요인

In [ ]:
print(user_title)

In [ ]:
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12)
SVD_matrix = SVD.fit_transform(user_title)
print(SVD_matrix)
# 이번에는 코사인 유사도말고, 상관계수를 사용
corr = np.corrcoef(SVD_matrix)

In [ ]:
movie_title = title_user.columns.tolist()
print(movie_title)
movie_index = movie_title.index("Avengers, The (2012)")
print(movie_index)
top_index = np.where(corr[movie_index] > 0.9)
for i in top_index[0]:
    print(movie_title[i])